In [1]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

len(df)
df.head()

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Activiti,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Activiti,3,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Activiti,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [3]:
df2 = df.copy()
df2

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Activiti,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Activiti,3,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Activiti,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,maxwell,6,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3166,maxwell,7,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3167,maxwell,8,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3168,maxwell,9,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


Agrupando as linhas para identificar quais os projetos que não possuem indício de BD
Grouping the lines to identify which projects that don't have DB clues

In [4]:
df2 = df2.groupby("PROJECTS").sum()
df2

,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,MariaDB,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
PROJECTS,,,,,,,,,,,,,,,,,,,,,
Activiti,55,0,0,0,0,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
AmazeFileManager,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Anki-Android,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeebe,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zeppelin,55,0,0,0,0,9,8,0,10,7,...,0,0,0,8,0,8,0,0,0,0
zipkin,55,0,0,0,0,0,0,0,0,7,...,0,0,0,9,0,0,0,0,0,0


Reseting the index column

In [5]:
df2 = df2.reset_index()
df2

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,55,0,0,0,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
1,AmazeFileManager,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Anki-Android,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AntennaPod,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,zeebe,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
313,zeppelin,55,0,0,0,0,9,8,0,10,...,0,0,0,8,0,8,0,0,0,0
314,zipkin,55,0,0,0,0,0,0,0,0,...,0,0,0,9,0,0,0,0,0,0
315,zookeeper,55,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


New dataframe filtering only lines that fifth version projects

In [6]:
#df10 = df2.mask(df2 > 1, 1)
df10 = df[(df['COMMITS'] == 5)]
df10

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14,airbyte,5,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
24,airsonic,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
34,spring-cloud-alibaba,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
44,nacos,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,yacy_search_server,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3134,YalpStore,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3144,SmartTubeNext,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3154,zaproxy,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Removing projects that don't have DB clues the principal dataframe 

In [7]:
#df2 = df2.iloc[ : , 1:]
#df3[(df3 == 0).all(axis=1)].index
df2 = df2.drop(df2[(df2.iloc[ : , 2:] == 0).all(axis=1)].index)
df2

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,55,0,0,0,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
2,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Arduino,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,AsciidocFX,55,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9,BIMserver,55,0,0,0,0,0,4,0,0,...,0,0,0,10,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,zaproxy,55,0,0,0,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
312,zeebe,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
313,zeppelin,55,0,0,0,0,9,8,0,10,...,0,0,0,8,0,8,0,0,0,0
314,zipkin,55,0,0,0,0,0,0,0,0,...,0,0,0,9,0,0,0,0,0,0


Removing projects that don't have DB clues in the fifth version 

In [8]:
df3 = df[(df['COMMITS'] == 5)]
#df3.drop(["PROJECTS", "COMMITS"], axis=1, inplace=True)
df3 = df3.drop(df3[(df3.iloc[:,2:] == 0).all(axis=1)].index) # deletando as linhas que possuem valor 0 em todas as colunas
df3

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14,airbyte,5,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
24,airsonic,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
34,spring-cloud-alibaba,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
44,nacos,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024,Saturn,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3084,wildfly,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3124,yacy_search_server,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3154,zaproxy,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Merging dataframe grouping and dataframe only last version with DB clues

In [9]:
#df3.merge(df4, on=['PROJECTS'], how='outer', suffixes=['', '_'], indicator=True)
df_mer= pd.merge(df2, df3, on= 'PROJECTS', how='outer', suffixes=['', '_'], indicator=True)
df_mer

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,PostGIS_,Etcd_,MySQL_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_MarkLogic-Sql_,Join_MarkLogic-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
0,Activiti,55,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,Arduino,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,AsciidocFX,55,0,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,BIMserver,55,0,0,0,0,0,4,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,zaproxy,55,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
193,zeebe,55,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
194,zeppelin,55,0,0,0,0,9,8,0,10,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,both
195,zipkin,55,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both


Filtering diference in the fifth version

In [10]:
df5 = df_mer[(df_mer['_merge'].str.startswith('left'))]
df5

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,PostGIS_,Etcd_,MySQL_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_MarkLogic-Sql_,Join_MarkLogic-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
1,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,Arduino,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,AsciidocFX,55,0,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,JSqlParser,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,NetGuard,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
15,OpenTripPlanner,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20,Shuttle,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,SpongeForge,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
27,alluxio,55,0,0,0,0,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29,ambry,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Merging dataframe only lines that fifth version projects and dataframe only lines in the fifth version and not DB clues

In [11]:
df_mer2= pd.merge(df10, df5.iloc[:,:1], on= 'PROJECTS', how='outer', suffixes=['', '_'], indicator=True)
df_mer2

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
0,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
1,airbyte,5,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,left_only
2,airsonic,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
3,spring-cloud-alibaba,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
4,nacos,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,yacy_search_server,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
313,YalpStore,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
314,SmartTubeNext,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
315,zaproxy,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only


Filtering only lines in the fifth version and not DB clues

In [12]:
df6 = df_mer2[(df_mer2['_merge'].str.startswith('bo'))]
df6

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
5,alluxio,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
11,cyclops,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
18,zookeeper,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
24,incubator-heron,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
37,iceberg,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
38,parquet-mr,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
39,bookkeeper,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
45,Arduino,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
46,AsciidocFX,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
65,Android-IMSI-Catcher-Detector,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both


Concatening dataframes

In [13]:
df_concat2 =  pd.concat([df3, df6]).drop_duplicates()
df_concat2

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
14,airbyte,5,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,NaN
24,airsonic,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
34,spring-cloud-alibaba,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
44,nacos,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,Shuttle,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
291,amidst,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
303,flexmark-java,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
305,dynmap,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both


In [ ]:
df2 = df2.astype(np.uint8)

In [14]:
df_concat2.drop(["PROJECTS","COMMITS","_merge"], axis=1, inplace=True)
df_concat2

,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,MariaDB,Neo4j,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
24,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
frq_items = apriori(df_concat2, min_support = 0.021, use_colnames = True)

In [16]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

In [17]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                             antecedents  \
76492                 (MySQL, Oracle, IBM DB2, Informix)   
67596  (SAP Adaptive Server, SQLite, Firebird, MS SQL...   
80757        (SAP Adaptive Server, Oracle, H2, Informix)   
39670                    (SAP Adaptive Server, Informix)   
67603       (SAP Adaptive Server, Firebird, IBM DB2, H2)   

                                          consequents  antecedent support  \
76492   (Firebird, SQLite, PostgreSQL, MS SQL Server)            0.025381   
67596                 (Oracle, IBM DB2, H2, Informix)            0.025381   
80757  (MySQL, SQLite, IBM DB2, Firebird, PostgreSQL)            0.025381   
39670   (Firebird, SQLite, PostgreSQL, MS SQL Server)            0.025381   
67603       (SQLite, Oracle, MS SQL Server, Informix)            0.025381   

       consequent support   support  confidence  lift  leverage  conviction  
76492            0.025381  0.025381         1.0  39.4  0.024737         inf  
67596            0.0

In [18]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
76492,"(MySQL, Oracle, IBM DB2, Informix)","(Firebird, SQLite, PostgreSQL, MS SQL Server)",0.025381,0.025381,0.025381,1.000000,39.400000,0.024737,inf
67596,"(SAP Adaptive Server, SQLite, Firebird, MS SQL...","(Oracle, IBM DB2, H2, Informix)",0.025381,0.025381,0.025381,1.000000,39.400000,0.024737,inf
80757,"(SAP Adaptive Server, Oracle, H2, Informix)","(MySQL, SQLite, IBM DB2, Firebird, PostgreSQL)",0.025381,0.025381,0.025381,1.000000,39.400000,0.024737,inf
39670,"(SAP Adaptive Server, Informix)","(Firebird, SQLite, PostgreSQL, MS SQL Server)",0.025381,0.025381,0.025381,1.000000,39.400000,0.024737,inf
67603,"(SAP Adaptive Server, Firebird, IBM DB2, H2)","(SQLite, Oracle, MS SQL Server, Informix)",0.025381,0.025381,0.025381,1.000000,39.400000,0.024737,inf
...,...,...,...,...,...,...,...,...,...
559,(Redis),"(MySQL, MS SQL Server)",0.152284,0.126904,0.025381,0.166667,1.313333,0.006055,1.047716
272,(MySQL),(Ehcache),0.411168,0.182741,0.096447,0.234568,1.283608,0.021309,1.067709
273,(Ehcache),(MySQL),0.182741,0.411168,0.096447,0.527778,1.283608,0.021309,1.246939
39,(Redis),(MS SQL Server),0.152284,0.131980,0.025381,0.166667,1.262821,0.005282,1.041624


In [19]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [20]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [21]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
182,(Firebird),(Informix),0.025381,0.030457,0.025381,1.000000,32.833333,0.024608,inf,1,1
183,(Informix),(Firebird),0.030457,0.025381,0.025381,0.833333,32.833333,0.024608,5.847716,1,1
192,(Firebird),(SAP Adaptive Server),0.025381,0.060914,0.025381,1.000000,16.416667,0.023835,inf,1,1
151,(SapHana),(SAP Adaptive Server),0.025381,0.060914,0.025381,1.000000,16.416667,0.023835,inf,1,1
193,(SAP Adaptive Server),(Firebird),0.060914,0.025381,0.025381,0.416667,16.416667,0.023835,1.670776,1,1
...,...,...,...,...,...,...,...,...,...,...,...
100,(H2),(Ehcache),0.243655,0.182741,0.060914,0.250000,1.368056,0.016388,1.089679,1,1
272,(MySQL),(Ehcache),0.411168,0.182741,0.096447,0.234568,1.283608,0.021309,1.067709,1,1
273,(Ehcache),(MySQL),0.182741,0.411168,0.096447,0.527778,1.283608,0.021309,1.246939,1,1
39,(Redis),(MS SQL Server),0.152284,0.131980,0.025381,0.166667,1.262821,0.005282,1.041624,1,1


In [22]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'seq_patterns/testes/historical_rules_commit5_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)